Plan of attack: 

-Subset THE DEMOGRAPHICS dataset by: 
Gender (Male/Female)
Race 

-Merge the demographics data and the Start FEIS data by patient ID # 
-Clean data so only relevant columns are left (Demographic data + family input)

We plan firstly to look at the spectrum of responses comparing available services/client mental health (as the answers are on a scale) and turn this into numerical data in order to quantify the quality of each subset’s degree of care.

We then plan to conduct topic modeling on the column in which families discuss where care is lacking in order to find the most popular/most desired methods of care that START did not provide. 

We also plan to conduct topic modeling and sentiment analysis on the column in which families offer advice to their caregiver in order to form a rough idea of the quality of care and how it may vary across demographic groups. We also are interested to see if these responses’ sentiment scores will trend in a specific direction, indicating biases in those who actually responded to the survey.


In [2]:
# Importing modules
import pandas as pd

In [24]:
demo_df = pd.read_excel(r"../files/Dartmouth_Data_Set.xlsx")
FEIS_df = pd.read_excel(r"../files/START_FEIS_Data.xlsx")
time_df = pd.read_excel(r"../files/Dartmouth_Time_Data.xlsx")
dict_df = pd.read_excel(r"../files/Final SIRS_Data_Dictionary_V13.1 October 2020.xlsx")

In [51]:
# Cleaning the demographics dataset
demographics = demo_df[['Local ID', 'Region', 'Date Enrolled in START', 'Gender', 'Race', 'Date of birth', 'Ethnicity',
                              'Level of Intellectual Disability', 'Psychiatric diagnoses', 'Medical diagnoses', 'Other Disabilities',
                              'Funding']]

In [69]:
# Merging datasets
merged = pd.merge(demographics, FEIS_df, how = 'inner', left_on = ['Local ID'], right_on = ['Respondent ID #  (SIRS Local ID)'])

# Look at type of join (changed)

merged['Local ID'].unique
merged.shape

(1097, 69)

In [64]:
# Subsetting by gender
demographics_male = merged.loc[merged['Gender']=='Male']
demographics_female = merged.loc[merged['Gender']=='Female']

# Subsetting by race
male_white = demographics_male[demographics_male['Race'] == "White"]
male_nonwhite = demographics_male[demographics_male['Race'] != "White"]

female_white = demographics_female[demographics_female['Race'] == "White"]
female_nonwhite = demographics_female[demographics_female['Race'] != "White"]


#male_white.shape
#male_nonwhite.shape

#female_white.shape
#female_nonwhite.shape



(114, 69)

In [ ]:
def sentiment_analyzer(test_str): 
    doj_entity = nlp(test_str)
    named_entities = [token.text for token in doj_entity.ents]
    
    fixed_entities = [re.escape(i) for i in named_entities]
    regex = "|".join(fixed_entities)
    delete_entities = re.sub(regex,"", test_str)
    
    sent_obj = SentimentIntensityAnalyzer()
    sentiment_example = sent_obj.polarity_scores(test_str)
    
    return sentiment_example

doj_sentiment = [sentiment_analyzer(entry) for entry in doj_subset["contents"]]